In [1]:
import pandas as pd
import numpy as np
import os
target_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\integral_keff\\data"
pre_process_data = ["int_exp_training_data.csv", "int_exp_validation_data.csv"]
void_value = 1.
variable_cassette_2a = True

In [2]:
os.chdir(target_directory)

In [3]:
#def encode(data):
#    print('Shape of data (BEFORE encode): %s' % str(data.shape))
#    encoded = to_categorical(data)
#    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
#    return encoded

In [4]:
#def one_hot_encode_fns(material_value, number_of_materials):
#    zeros_ = np.zeros(number_of_materials)
#    material_array = zeros_
#    material_array[int(material_value)-1] = 1
#    print(material_array)
#one_hot_encode_fns(material_value=4, number_of_materials = 4)

In [5]:
def preprocess_variable_cassette_raw_ga_data(data_file, output_file_name="", fixed_plate_count=120):
    with open(data_file, 'r') as data_file_obj:
        data_master_list = []
        for line in data_file_obj:
            list_ = line.split(",")

            # print(len(list_))
            # print(list_)
            new_list = []
            for value in list_:
                if value.strip() == "":
                    continue
                new_list.append(value.strip())
            # print(len(new_list), len(new_list) - fixed_plate_count - 1)
            # print(new_list)
            size_of_all_patterns = len(new_list) - 1

            size_of_variable_cassette = size_of_all_patterns - fixed_plate_count

            new_list_with_variable_cassette = []
            for count, value in enumerate(new_list):
                new_list_with_variable_cassette.append(value)
                if count == 120:
                    for _ in range(30 - size_of_variable_cassette):
                        new_list_with_variable_cassette.append('1')
            # print("final list", len(new_list_with_variable_cassette), new_list_with_variable_cassette)
            data_master_list.append(new_list_with_variable_cassette)
        # print(len(data_master_list))
        with open(output_file_name, 'w') as output_file_obj:
            for _ in data_master_list:
                write_string = ""
                for __ in _:
                    write_string +=  str(__) + ","
                output_file_obj.write(write_string[:-1] + "\n")

In [6]:
for preproc_ in pre_process_data:
    preprocess_variable_cassette_raw_ga_data(data_file = preproc_, output_file_name = "processed_" + preproc_)

In [7]:
def return_pattern(pat_a, pat_b, pat_c, plate_number):
    row_1 = np.array([pat_c[plate_number], pat_b[plate_number],
                     pat_b[plate_number], pat_a[plate_number]])
    return np.reshape(row_1, (2,2))


In [8]:
#data_ = np.loadtxt("rep_validation_gen_46.csv", delimiter=",")
#X = data_[:,0:150]
#Y = data_[:,150:151]
#zones = [1, 2, 3]
#number_of_plates = 60

def build_3x3_fns_data(data_file_string, zones = [1, 2, 3], number_of_plates = 60, number_of_materials = 4):
    data_ = np.loadtxt(data_file_string, delimiter=",")
    X = data_[:,1:151]
    Y = data_[:,0:1]
    print(X[0])
    print(Y[0])
    pattern_shapes_list = []
    for fns_pattern_count, pattern in enumerate(X): 

        ### Setting patterns for this dataset
        if variable_cassette_2a == False:
            pattern_1a = pattern[0:10]
            pattern_1b = pattern[10:30]
            pattern_1c = pattern[30:50]
            pattern_2a = pattern[50:70]
            pattern_2b = pattern[70:90]
            pattern_2c = pattern[90:110]
            pattern_3a = np.array([void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value])
            pattern_3b = pattern[110:130]
            pattern_3c = pattern[130:150]

            np_four_10_array = np.array([void_value, void_value, void_value, void_value, void_value,
                                         void_value, void_value, void_value, void_value, void_value])
            pattern_1a = np.concatenate((np_four_10_array, pattern_1a))
        
        else:
            pattern_1b = pattern[0:20]
            pattern_1c = pattern[20:40]
            pattern_2b = pattern[40:60]
            pattern_2c = pattern[60:80]

            pattern_3b = pattern[80:100]
            pattern_3c = pattern[100:120]

            np_four_10_array = np.array([void_value, void_value, void_value, void_value, void_value,
                                         void_value, void_value, void_value, void_value, void_value])
            pattern_1a = pattern[120:130]
            pattern_1a = np.concatenate((np_four_10_array, pattern_1a))
            pattern_2a = pattern[130:150]
            pattern_3a = np.array([void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value])

        for plate in range(number_of_plates):
            if plate == 0:
                pattern_array = return_pattern(pattern_1a, pattern_1b, pattern_1c, plate)
                continue

            if plate < 20:
                pattern_ = return_pattern(pattern_1a, pattern_1b, pattern_1c, plate)
                pattern_array = np.concatenate((pattern_array, pattern_))
            if plate < 40 and plate >= 20:
                pattern_ = return_pattern(pattern_2a, pattern_2b, pattern_2c, plate-20)
                pattern_array = np.concatenate((pattern_array, pattern_))
            if plate >= 40:
                pattern_ = return_pattern(pattern_3a, pattern_3b, pattern_3c, plate-40)
                pattern_array = np.concatenate((pattern_array, pattern_))

        pattern_array_reshaped =  np.reshape(pattern_array, (60, 2, 2))
        #print(pattern_array_reshaped.shape)
        ### One hot encoding the 3x3x60 array into a 3x3x60x5 array. Stackoverflow magic:
        ### https://stackoverflow.com/questions/36960320/convert-a-2d-matrix-to-a-3d-one-hot-matrix-numpy
        ### 4 types of materials is hard coded inside np.arange(#)
        pattern_array_reshaped_1he = (np.arange(number_of_materials) == pattern_array_reshaped[...,None]-1).astype(int)

        pattern_shapes_list.append(np.array(pattern_array_reshaped_1he))
        #pattern_array_reshaped_1he_array = np.append(pattern_array_reshaped_1he_array, pattern_array_reshaped_1he)
        print(fns_pattern_count, pattern_array_reshaped_1he.shape)
    pattern_shapes_array = np.array(pattern_shapes_list)
    return pattern_shapes_array, Y

In [9]:
for data in pre_process_data:
    
    pattern_shapes_array, Y = build_3x3_fns_data("processed_"+data)
    np.save("X_"+data, pattern_shapes_array)
    np.save("Y_"+data, Y)

[3. 4. 2. 3. 3. 2. 2. 4. 4. 2. 2. 2. 4. 2. 3. 4. 2. 2. 2. 4. 2. 3. 4. 3.
 2. 2. 4. 2. 3. 2. 4. 4. 4. 4. 4. 4. 4. 3. 2. 4. 3. 2. 3. 4. 4. 4. 3. 4.
 2. 2. 4. 4. 3. 2. 4. 4. 4. 4. 2. 2. 4. 3. 3. 3. 4. 4. 4. 4. 3. 4. 3. 4.
 4. 3. 4. 3. 2. 3. 4. 2. 3. 4. 3. 4. 3. 2. 4. 4. 2. 4. 4. 4. 4. 3. 2. 4.
 2. 3. 4. 2. 4. 3. 3. 3. 4. 3. 2. 2. 4. 2. 3. 4. 4. 4. 2. 4. 2. 4. 4. 2.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 2.]
[-0.01982]
0 (60, 2, 2, 4)
1 (60, 2, 2, 4)
2 (60, 2, 2, 4)
3 (60, 2, 2, 4)
4 (60, 2, 2, 4)
5 (60, 2, 2, 4)
6 (60, 2, 2, 4)
7 (60, 2, 2, 4)
8 (60, 2, 2, 4)
9 (60, 2, 2, 4)
10 (60, 2, 2, 4)
11 (60, 2, 2, 4)
12 (60, 2, 2, 4)
13 (60, 2, 2, 4)
14 (60, 2, 2, 4)
15 (60, 2, 2, 4)
16 (60, 2, 2, 4)
17 (60, 2, 2, 4)
18 (60, 2, 2, 4)
19 (60, 2, 2, 4)
20 (60, 2, 2, 4)
21 (60, 2, 2, 4)
22 (60, 2, 2, 4)
23 (60, 2, 2, 4)
24 (60, 2, 2, 4)
25 (60, 2, 2, 4)
26 (60, 2, 2, 4)
27 (60, 2, 2, 4)
28 (60, 2, 2, 4)
29 (60, 2, 2, 4)
30 (60, 2, 2, 4)
31 (60, 2, 2, 

477 (60, 2, 2, 4)
478 (60, 2, 2, 4)
479 (60, 2, 2, 4)
480 (60, 2, 2, 4)
481 (60, 2, 2, 4)
482 (60, 2, 2, 4)
483 (60, 2, 2, 4)
484 (60, 2, 2, 4)
485 (60, 2, 2, 4)
486 (60, 2, 2, 4)
487 (60, 2, 2, 4)
488 (60, 2, 2, 4)
489 (60, 2, 2, 4)
490 (60, 2, 2, 4)
491 (60, 2, 2, 4)
492 (60, 2, 2, 4)
493 (60, 2, 2, 4)
494 (60, 2, 2, 4)
495 (60, 2, 2, 4)
496 (60, 2, 2, 4)
497 (60, 2, 2, 4)
498 (60, 2, 2, 4)
499 (60, 2, 2, 4)
500 (60, 2, 2, 4)
501 (60, 2, 2, 4)
502 (60, 2, 2, 4)
503 (60, 2, 2, 4)
504 (60, 2, 2, 4)
505 (60, 2, 2, 4)
506 (60, 2, 2, 4)
507 (60, 2, 2, 4)
508 (60, 2, 2, 4)
509 (60, 2, 2, 4)
510 (60, 2, 2, 4)
511 (60, 2, 2, 4)
512 (60, 2, 2, 4)
513 (60, 2, 2, 4)
514 (60, 2, 2, 4)
515 (60, 2, 2, 4)
516 (60, 2, 2, 4)
517 (60, 2, 2, 4)
518 (60, 2, 2, 4)
519 (60, 2, 2, 4)
520 (60, 2, 2, 4)
521 (60, 2, 2, 4)
522 (60, 2, 2, 4)
523 (60, 2, 2, 4)
524 (60, 2, 2, 4)
525 (60, 2, 2, 4)
526 (60, 2, 2, 4)
527 (60, 2, 2, 4)
528 (60, 2, 2, 4)
529 (60, 2, 2, 4)
530 (60, 2, 2, 4)
531 (60, 2, 2, 4)
532 (60, 2

956 (60, 2, 2, 4)
957 (60, 2, 2, 4)
958 (60, 2, 2, 4)
959 (60, 2, 2, 4)
960 (60, 2, 2, 4)
961 (60, 2, 2, 4)
962 (60, 2, 2, 4)
963 (60, 2, 2, 4)
964 (60, 2, 2, 4)
965 (60, 2, 2, 4)
966 (60, 2, 2, 4)
967 (60, 2, 2, 4)
968 (60, 2, 2, 4)
969 (60, 2, 2, 4)
970 (60, 2, 2, 4)
971 (60, 2, 2, 4)
972 (60, 2, 2, 4)
973 (60, 2, 2, 4)
974 (60, 2, 2, 4)
975 (60, 2, 2, 4)
976 (60, 2, 2, 4)
977 (60, 2, 2, 4)
978 (60, 2, 2, 4)
979 (60, 2, 2, 4)
980 (60, 2, 2, 4)
981 (60, 2, 2, 4)
982 (60, 2, 2, 4)
983 (60, 2, 2, 4)
984 (60, 2, 2, 4)
985 (60, 2, 2, 4)
986 (60, 2, 2, 4)
987 (60, 2, 2, 4)
988 (60, 2, 2, 4)
989 (60, 2, 2, 4)
990 (60, 2, 2, 4)
991 (60, 2, 2, 4)
992 (60, 2, 2, 4)
993 (60, 2, 2, 4)
994 (60, 2, 2, 4)
995 (60, 2, 2, 4)
996 (60, 2, 2, 4)
997 (60, 2, 2, 4)
998 (60, 2, 2, 4)
999 (60, 2, 2, 4)
1000 (60, 2, 2, 4)
1001 (60, 2, 2, 4)
1002 (60, 2, 2, 4)
1003 (60, 2, 2, 4)
1004 (60, 2, 2, 4)
1005 (60, 2, 2, 4)
1006 (60, 2, 2, 4)
1007 (60, 2, 2, 4)
1008 (60, 2, 2, 4)
1009 (60, 2, 2, 4)
1010 (60, 2, 2, 4)

1412 (60, 2, 2, 4)
1413 (60, 2, 2, 4)
1414 (60, 2, 2, 4)
1415 (60, 2, 2, 4)
1416 (60, 2, 2, 4)
1417 (60, 2, 2, 4)
1418 (60, 2, 2, 4)
1419 (60, 2, 2, 4)
1420 (60, 2, 2, 4)
1421 (60, 2, 2, 4)
1422 (60, 2, 2, 4)
1423 (60, 2, 2, 4)
1424 (60, 2, 2, 4)
1425 (60, 2, 2, 4)
1426 (60, 2, 2, 4)
1427 (60, 2, 2, 4)
1428 (60, 2, 2, 4)
1429 (60, 2, 2, 4)
1430 (60, 2, 2, 4)
1431 (60, 2, 2, 4)
1432 (60, 2, 2, 4)
1433 (60, 2, 2, 4)
1434 (60, 2, 2, 4)
1435 (60, 2, 2, 4)
1436 (60, 2, 2, 4)
1437 (60, 2, 2, 4)
1438 (60, 2, 2, 4)
1439 (60, 2, 2, 4)
1440 (60, 2, 2, 4)
1441 (60, 2, 2, 4)
1442 (60, 2, 2, 4)
1443 (60, 2, 2, 4)
1444 (60, 2, 2, 4)
1445 (60, 2, 2, 4)
1446 (60, 2, 2, 4)
1447 (60, 2, 2, 4)
1448 (60, 2, 2, 4)
1449 (60, 2, 2, 4)
1450 (60, 2, 2, 4)
1451 (60, 2, 2, 4)
1452 (60, 2, 2, 4)
1453 (60, 2, 2, 4)
1454 (60, 2, 2, 4)
1455 (60, 2, 2, 4)
1456 (60, 2, 2, 4)
1457 (60, 2, 2, 4)
1458 (60, 2, 2, 4)
1459 (60, 2, 2, 4)
1460 (60, 2, 2, 4)
1461 (60, 2, 2, 4)
1462 (60, 2, 2, 4)
1463 (60, 2, 2, 4)
1464 (60, 2,

1858 (60, 2, 2, 4)
1859 (60, 2, 2, 4)
1860 (60, 2, 2, 4)
1861 (60, 2, 2, 4)
1862 (60, 2, 2, 4)
1863 (60, 2, 2, 4)
1864 (60, 2, 2, 4)
1865 (60, 2, 2, 4)
1866 (60, 2, 2, 4)
1867 (60, 2, 2, 4)
1868 (60, 2, 2, 4)
1869 (60, 2, 2, 4)
1870 (60, 2, 2, 4)
1871 (60, 2, 2, 4)
1872 (60, 2, 2, 4)
1873 (60, 2, 2, 4)
1874 (60, 2, 2, 4)
1875 (60, 2, 2, 4)
1876 (60, 2, 2, 4)
1877 (60, 2, 2, 4)
1878 (60, 2, 2, 4)
1879 (60, 2, 2, 4)
1880 (60, 2, 2, 4)
1881 (60, 2, 2, 4)
1882 (60, 2, 2, 4)
1883 (60, 2, 2, 4)
1884 (60, 2, 2, 4)
1885 (60, 2, 2, 4)
1886 (60, 2, 2, 4)
1887 (60, 2, 2, 4)
1888 (60, 2, 2, 4)
1889 (60, 2, 2, 4)
1890 (60, 2, 2, 4)
1891 (60, 2, 2, 4)
1892 (60, 2, 2, 4)
1893 (60, 2, 2, 4)
1894 (60, 2, 2, 4)
1895 (60, 2, 2, 4)
1896 (60, 2, 2, 4)
1897 (60, 2, 2, 4)
1898 (60, 2, 2, 4)
1899 (60, 2, 2, 4)
1900 (60, 2, 2, 4)
1901 (60, 2, 2, 4)
1902 (60, 2, 2, 4)
1903 (60, 2, 2, 4)
1904 (60, 2, 2, 4)
1905 (60, 2, 2, 4)
1906 (60, 2, 2, 4)
1907 (60, 2, 2, 4)
1908 (60, 2, 2, 4)
1909 (60, 2, 2, 4)
1910 (60, 2,

2311 (60, 2, 2, 4)
2312 (60, 2, 2, 4)
2313 (60, 2, 2, 4)
2314 (60, 2, 2, 4)
2315 (60, 2, 2, 4)
2316 (60, 2, 2, 4)
2317 (60, 2, 2, 4)
2318 (60, 2, 2, 4)
2319 (60, 2, 2, 4)
2320 (60, 2, 2, 4)
2321 (60, 2, 2, 4)
2322 (60, 2, 2, 4)
2323 (60, 2, 2, 4)
2324 (60, 2, 2, 4)
2325 (60, 2, 2, 4)
2326 (60, 2, 2, 4)
2327 (60, 2, 2, 4)
2328 (60, 2, 2, 4)
2329 (60, 2, 2, 4)
2330 (60, 2, 2, 4)
2331 (60, 2, 2, 4)
2332 (60, 2, 2, 4)
2333 (60, 2, 2, 4)
2334 (60, 2, 2, 4)
2335 (60, 2, 2, 4)
2336 (60, 2, 2, 4)
2337 (60, 2, 2, 4)
2338 (60, 2, 2, 4)
2339 (60, 2, 2, 4)
2340 (60, 2, 2, 4)
2341 (60, 2, 2, 4)
2342 (60, 2, 2, 4)
2343 (60, 2, 2, 4)
2344 (60, 2, 2, 4)
2345 (60, 2, 2, 4)
2346 (60, 2, 2, 4)
2347 (60, 2, 2, 4)
2348 (60, 2, 2, 4)
2349 (60, 2, 2, 4)
2350 (60, 2, 2, 4)
2351 (60, 2, 2, 4)
2352 (60, 2, 2, 4)
2353 (60, 2, 2, 4)
2354 (60, 2, 2, 4)
2355 (60, 2, 2, 4)
2356 (60, 2, 2, 4)
2357 (60, 2, 2, 4)
2358 (60, 2, 2, 4)
2359 (60, 2, 2, 4)
2360 (60, 2, 2, 4)
2361 (60, 2, 2, 4)
2362 (60, 2, 2, 4)
2363 (60, 2,

2813 (60, 2, 2, 4)
2814 (60, 2, 2, 4)
2815 (60, 2, 2, 4)
2816 (60, 2, 2, 4)
2817 (60, 2, 2, 4)
2818 (60, 2, 2, 4)
2819 (60, 2, 2, 4)
2820 (60, 2, 2, 4)
2821 (60, 2, 2, 4)
2822 (60, 2, 2, 4)
2823 (60, 2, 2, 4)
2824 (60, 2, 2, 4)
2825 (60, 2, 2, 4)
2826 (60, 2, 2, 4)
2827 (60, 2, 2, 4)
2828 (60, 2, 2, 4)
2829 (60, 2, 2, 4)
2830 (60, 2, 2, 4)
2831 (60, 2, 2, 4)
2832 (60, 2, 2, 4)
2833 (60, 2, 2, 4)
2834 (60, 2, 2, 4)
2835 (60, 2, 2, 4)
2836 (60, 2, 2, 4)
2837 (60, 2, 2, 4)
2838 (60, 2, 2, 4)
2839 (60, 2, 2, 4)
2840 (60, 2, 2, 4)
2841 (60, 2, 2, 4)
2842 (60, 2, 2, 4)
2843 (60, 2, 2, 4)
2844 (60, 2, 2, 4)
2845 (60, 2, 2, 4)
2846 (60, 2, 2, 4)
2847 (60, 2, 2, 4)
2848 (60, 2, 2, 4)
2849 (60, 2, 2, 4)
2850 (60, 2, 2, 4)
2851 (60, 2, 2, 4)
2852 (60, 2, 2, 4)
2853 (60, 2, 2, 4)
2854 (60, 2, 2, 4)
2855 (60, 2, 2, 4)
2856 (60, 2, 2, 4)
2857 (60, 2, 2, 4)
2858 (60, 2, 2, 4)
2859 (60, 2, 2, 4)
2860 (60, 2, 2, 4)
2861 (60, 2, 2, 4)
2862 (60, 2, 2, 4)
2863 (60, 2, 2, 4)
2864 (60, 2, 2, 4)
2865 (60, 2,